# State Machine Cleaned -- with whole Steps

## Çift Terimden Tek Terime Geçiş 


### Step 1: Transition Process Control
- **State Number**: 2
- **State Code**: 10340002
- **State Name**: Transition Process from Dual to Single Term Initiated
- **Description**: The transition process is controlled here. Five different checks are performed; if all checks are successful, it moves to the next state; otherwise, it goes to the termination state.


#### 1a. Çalışma Öncesi Script

In [10]:
# Yok 

#### 1b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do business/mapping here
    return {"body": {}, "params": {"assetId":str(request_data["id"])}}


#### 1c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {}
    
    attribute_codes = ['ServicePointId', 'BusinessPartnerId', 'ContractAccountId']
    
    for code in attribute_codes:
        for attr in request_data['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # values içindeki value al
                store[code] = value
                break
        else:
            store[code] = None  # Eğer bulunamazsa None döner
    return store

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {}
    
    attribute_codes = ['ServicePointId', 'BusinessPartnerId', 'ContractAccountId']
    current_state_response = state_responses_of_machine['x']['businessLogic']
    for code in attribute_codes:
        for attr in current_state_response['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # values içindeki value al
                store[code] = value
                break
        else:
            store[code] = None  # Eğer bulunamazsa None döner
    return store

#### 1d. Çalışma Sonrası Scripti

In [ ]:

# A service will be added here to check the historical data count of TermType. If this stage has been reached, it indicates that the number of Term Changes is less than 3.

# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}

def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    condition = {
        'TermType': 'CT',
        'ContractCancellationDate': 99991231235959
    }

    if request_data['assetDefinitionCode'] not in ['ConsumerContract', 'LicensedProducerContract',
                                              'UnlicensedProducerContract']:
        return {"success": True, "error": {"code": "END", "detail": "Tüketici değil iseniz terim değişikliği yapılamaz"}}

    # ConsumerContract için TariffCode kontrolü
    if request_data['assetDefinitionCode'] == 'ConsumerContract':
        tariff_code = next((attr['value'] for attr in current_state_response['attributes']
                            if attr['attribute']['attributeCode'] == 'VoltageType'), None)['values'][0]['value']

        # Eğer tariff_code None ise, values'dan al
        if tariff_code == 'AG':
                return {"success": True, "error": {"code": "END", "detail": "AG iseniz terim değişikliği yapılamaz"}}

    # Şimdi kalan kontrollerde sıra
    if all(next((attr['value'] if attr['value'] is not None else (attr['values'][0]['value'] if attr['values'] else None)
        for attr in request_data['attributes']
            if attr['attribute']['attributeCode'] == key), None) == value for key, value in condition.items()):
                return {"success": True, "goto": "10340003"}
    else:
        return {"success": True, "error": {"code": "END", "detail": "Terim değişikliği yapmaya uygun değilsiniz"}}


## Step 2: Invoice Creation
- **State Number**: 3
- **State Code**: 10340003
- **State Name**: Invoice Creation
- **Description**: After the machine is created, it waits for the first invoice to be generated. At this stage, a specific event (from Kafka) is triggered.
- **Status**: Pending

#### 2a. Çalışma Öncesi Scripti

#### 2b. İstek Eşleştirme Scripti 

#### 2c. State Context Değer Tutma Scripti

#### 2d. Çalışma Sonrası Scripti

## Step 3: Contract Cancellation
- **State Number**: 4
- **State Code**: 10340004
- **State Name**: Contract Cancellation
- **Description**: This state signifies the Cancellation of the existing dual-term contract. Relevant date information is updated here, and then it transitions to the next state.

#### 3a. Çalışma Öncesi Scripti

In [19]:
#Yok

#### 3b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_responses_of_machine):
    
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    
    data = {
        "id": request_data["id"],
        "attributes": [
            {
                "attribute": {
                    "attributeCode": "ContractCancellationDate"
                },
                "value":current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationRequestDate"
                },
                "value": current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationReason"
                },
                "value": "SubscriberRequest"  
            },
            {
                "attribute": {
                    "attributeCode": "ContractNumber"
                },
                "value": next((attr['value'] for attr in request_data['attributes'] 
                               if attr['attribute']['attributeCode'] == "ContractNumber"), None)
            }
        ],
        "authorityGroup": request_data["authorityGroup"],
        "contractLockID": "string"  
    }
    
    return {"body": data}


#### 3c. State Context Değer Tutma Scripti

In [20]:
## Yok

#### 3d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "10340005"}



## Step 4: Closure Accrual Creation
- **State Number**: 5
- **State Code**: 10340005
- **State Name**: Closure Accrual Creation
- **Description**: A closure invoice is created for the terminated contract. Here, the invoice creation dates are processed.
- **Status**: Pending

#### 4a. Çalışma Öncesi Scripti

In [21]:
# Yok

#### 4b. İstek Eşleştirme Scripti 

In [22]:
#yok

#### 4c. State Context Değer Tutma Scripti

In [23]:
#Yok

#### 4d. Çalışma Sonrası Scripti

In [24]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    # do workflow here
    return {"success": True, "goto": "10340006-1"}


#### 4e. İş Mantığı

In [25]:
"""http://thor-stage.edas1.com/host/v1//accruals/calculate"""

'http://thor-stage.edas1.com/host/v1//accruals/calculate'

## Step 5: Data Preparation for New Account
- **State Number**: -
- **State Code**: -
- **State Name**: Data Preparation for New Account
- **Description**: This process involves preparing the necessary data for creating a new account. Relevant attributes are queried.
- **Status**: Pending

#### 5a. Çalışma Öncesi Scripti

#### 5b. İstek Eşleştirme Scripti 

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, businessLogic, webhook}>
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):  
    contractAccountId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='ContractAccountId'),None)['value']

    return {"params": {"assetId":contractAccountId}}

#### 5c. State Context Değer Tutma Scripti

In [27]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, prev_state_response):
    store = {request_data}
    return store

#### 5d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "10340006"}


#### 5e. İş Mantığı

In [31]:
"""http://thor-stage.edas1.com/host/v1//md/search/{assetId}"""##URL is  not defined

'http://thor-stage.edas1.com/host/v1//md/search/{assetId}'

## Step 6: Create New Account
- **State Number**: 6
- **State Code**: 10340006
- **State Name**: Create New Account
- **Description**: An account is created to link with the new contract. New account definitions are made using existing attributes.
- **Status** : Pending

#### 6a. Çalışma Öncesi Scripti

In [32]:
#Yok

#### 6b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_response_of_machine):
    prev_state_response = state_response_of_machine

    current_date = datetime.now().strftime('%Y%m%d%H%m%S')
    #accountCreateDate = next((attr for attr in prev_state_response['attributes'] if attr['attribute']['attributeCode'] == 'CreateDate'),None)['value']

    deletelist = ['AssetIdentifier']
    prev_state_response


    copied_json = {}
    stack = [(copied_json, prev_state_response)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    for attr in copied_json['attributes']:
        if attr['attribute']['attributeCode']=='ExpiryDate':
            attr['value'] = current_date

    copied_json['attributes'] = [attr for attr in copied_json['attributes'] if attr['attribute']['attributeCode'] not in deletelist]

    
    
    return copied_json


#### 6c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {}
    store ['NewAccountId'] = state_responses_of_machine['10340006']['businessLogicResponse']['id'] 
    return store


#### 6d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "10340007"}


#### 6e. İş Mantığı

In [37]:
"""http://thor-stage.edas1.com/host/v1//account/create"""


'http://thor-stage.edas1.com/host/v1//account/create'

## Step 7: Creation of New Contract
- **State Number**: 7
- **State Code**: 10340007
- **State Name**: Creation of New Contract
- **Description**: A new contract is created with the new term information. The same attributes as the existing contract are used to create a single-term contract.

#### 7a. Çalışma Öncesi Scripti

In [38]:
#Yok

#### 7b. İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, prev_state_response, current_state_response) -> json:
    attribute_codes = ['ServicePointId', 'BusinessPartnerId', 'AccountId']
    json_parts = []

    for code in attribute_codes:
        value = None  # Default value
        
        for attr in request_data['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # Get value from values if available
                break
        
        json_parts.append(f'"{code}": {json.dumps(value)}')  # Create JSON key-value pairs
    
    json_output = '{' + ', '.join(json_parts) + '}'
    return json_output  # Return the JSON 



##### 7b. İstek Eşleştirme Scripti- Olması gereken(2)

In [ ]:
from datetime import datetime
import json
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # Not recursive 
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    
    
    changes_dict = {'TermType': 'TT',
                    'ContractAccountId': context['accountInfo']['id']
                    }
    
    copied_json = {}
    stack = [(copied_json, request_data)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959

    # Update attribute values in context according to the dict above
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['valueDesc'] = None
                    attr['value'] = new_value

    return {"body":copied_json}  

#### 7c. State Context Değer Tutma Scripti

In [39]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, prev_state_response):
    store = {}
    
    attribute_codes = ['ContractId']
    
    for code in attribute_codes:
        for attr in request_data['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # values içindeki value al
                store[code] = value
                break
        else:
            store[code] = None  # Eğer bulunamazsa None döner
    return store

##### 7d. Çalışma Sonrası Scripti

In [42]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    # do workflow here
    return {"success": True, "goto": "END"}


#### 7e. İş Mantığı

In [41]:
"""http://thor-stage.edas1.com/host/v1//contract/create"""

'http://thor-stage.edas1.com/host/v1//contract/create'